In [1]:
import pandas as pd
import requests

In [2]:
def anho(y):
    print(" ")
    key = "otqvshn22qrt"
    regionCode = "CL"
    payload={}
    headers = {
        'X-eBirdApiToken': key
    }
    df = pd.DataFrame()
    for m in range(1, 13):
        for d in range(1, 32):
            print(f"{d}-{m}-{y}", end=" ; ")
            url = url = f"https://api.ebird.org/v2/data/obs/{regionCode}/historic/{y}/{m}/{d}"
            response = requests.request("GET", url, headers=headers, data=payload)
            df = pd.concat([df, pd.json_normalize(response.json())], ignore_index=True)
    return df

In [3]:
for y in range(2021, 2023):
    df = anho(y)
    df.to_csv(f"vis_{y}.csv", index=False)

 
1-1-2021 ; 2-1-2021 ; 3-1-2021 ; 4-1-2021 ; 5-1-2021 ; 6-1-2021 ; 7-1-2021 ; 8-1-2021 ; 9-1-2021 ; 10-1-2021 ; 11-1-2021 ; 12-1-2021 ; 13-1-2021 ; 14-1-2021 ; 15-1-2021 ; 16-1-2021 ; 17-1-2021 ; 18-1-2021 ; 19-1-2021 ; 20-1-2021 ; 21-1-2021 ; 22-1-2021 ; 23-1-2021 ; 24-1-2021 ; 25-1-2021 ; 26-1-2021 ; 27-1-2021 ; 28-1-2021 ; 29-1-2021 ; 30-1-2021 ; 31-1-2021 ; 1-2-2021 ; 2-2-2021 ; 3-2-2021 ; 4-2-2021 ; 5-2-2021 ; 6-2-2021 ; 7-2-2021 ; 8-2-2021 ; 9-2-2021 ; 10-2-2021 ; 11-2-2021 ; 12-2-2021 ; 13-2-2021 ; 14-2-2021 ; 15-2-2021 ; 16-2-2021 ; 17-2-2021 ; 18-2-2021 ; 19-2-2021 ; 20-2-2021 ; 21-2-2021 ; 22-2-2021 ; 23-2-2021 ; 24-2-2021 ; 25-2-2021 ; 26-2-2021 ; 27-2-2021 ; 28-2-2021 ; 29-2-2021 ; 30-2-2021 ; 31-2-2021 ; 1-3-2021 ; 2-3-2021 ; 3-3-2021 ; 4-3-2021 ; 5-3-2021 ; 6-3-2021 ; 7-3-2021 ; 8-3-2021 ; 9-3-2021 ; 10-3-2021 ; 11-3-2021 ; 12-3-2021 ; 13-3-2021 ; 14-3-2021 ; 15-3-2021 ; 16-3-2021 ; 17-3-2021 ; 18-3-2021 ; 19-3-2021 ; 20-3-2021 ; 21-3-2021 ; 22-3-2021 ; 23-3-2021 ; 24-3-

In [2]:
df_2019 = pd.read_csv("vis_2019.csv")
df_2020 = pd.read_csv("vis_2020.csv")

In [47]:
df_2019

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId,exoticCategory,errors
0,soulap1,Southern Lapwing,Vanellus chilensis,L4026975,Loma Linda,2019-01-01 23:00,1.0,-33.435764,-71.677103,True,False,False,S51197645,NaN,NaN
1,brnowl,Barn Owl,Tyto alba,L4026975,Loma Linda,2019-01-01 23:00,1.0,-33.435764,-71.677103,True,False,False,S51197645,NaN,NaN
2,aupowl1,Austral Pygmy-Owl,Glaucidium nana,L4026975,Loma Linda,2019-01-01 23:00,1.0,-33.435764,-71.677103,True,False,False,S51197645,NaN,NaN
3,austhr1,Austral Thrush,Turdus falcklandii,L7488647,Plaza Melvin Jones,2019-01-01 22:25,1.0,-33.451996,-70.593581,True,False,False,S51194229,NaN,NaN
4,whcela1,White-crested Elaenia,Elaenia albiceps,L8346445,"1485 Loncoyen, Valdivia, Región de los Ríos, C...",2019-01-01 22:05,NaN,-39.824130,-73.278223,True,False,True,S51192610,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62676,verfly,Vermilion Flycatcher,Pyrocephalus rubinus,L760571,Azapa,2019-12-31 08:10,2.0,-18.517247,-70.197006,True,False,False,S63139126,NaN,NaN
62677,slbfin3,Slender-billed Finch,Xenospingus concolor,L760571,Azapa,2019-12-31 08:10,1.0,-18.517247,-70.197006,True,False,False,S63139126,NaN,NaN
62678,blbgra1,Blue-black Grassquit,Volatinia jacarina,L760571,Azapa,2019-12-31 08:10,1.0,-18.517247,-70.197006,True,False,False,S63139126,NaN,NaN
62679,chtsee1,Chestnut-throated Seedeater,Sporophila telasco,L760571,Azapa,2019-12-31 08:10,10.0,-18.517247,-70.197006,True,False,False,S63139126,NaN,NaN


In [4]:
ubicaciones = df_2019[["locName", "lat", "lng"]].drop_duplicates(df_2019.columns[df_2019.columns.isin(['locName'])],
                        keep='first')
ubicaciones = ubicaciones.dropna()

In [5]:
ubicaciones

,locName,lat,lng
0,Loma Linda,-33.435764,-71.677103
3,Plaza Melvin Jones,-33.451996,-70.593581
4,"1485 Loncoyen, Valdivia, Región de los Ríos, C...",-39.824130,-73.278223
5,"(-35.594, -71.735)",-35.593632,-71.735010
6,Poniente--Calles y Plazas,-33.537781,-70.561735
...,...,...,...
62618,Laguna Rivera,-39.344536,-71.688309
62634,Dientes de Navarino--Laguna Betinelli,-55.047450,-67.628789
62655,Faro San Isidro--Cruz de froward,-53.832784,-71.079483
62657,"Pinto, Ñuble, CL (-36.878, -71.464)",-36.878458,-71.463928


In [29]:
#https://nominatim.org/release-docs/develop/api/Overview/
def region(lat, lon):
    url = f'https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json'
    page = requests.get(url)
    json = page.json()
    try:
        if json['address']['country'] == "Argentina":
            return "Argentina"
        else:
            return json['address']['state']
    except:
        return None

In [30]:
ubicaciones["state"] = 0

In [32]:
ubicaciones['state'] = ubicaciones.apply(lambda x: region(x['lat'], x['lng']), axis=1)
ubicaciones

In [4]:
import geopandas as gpd
regiones = gpd.read_file("Regional.shp", driver="SHP")
regiones

DriverError: Unable to open Regional.shx or Regional.SHX. Set SHAPE_RESTORE_SHX config option to YES to restore or create it.